<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/info_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!pip install pyahocorasick
!pip install fuzzyset
from ahocorasick import Automaton
import fuzzyset
import urllib
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import re
import pandas as pd
import numpy as np
nltk.download('punkt')
!pip install wget
!pip install spacy
!pip install regex
!python -m spacy download en_core_web_sm
import spacy
import regex

import os
import wget
colabimporturl = 'https://github.com/ratmcu/colaboratory_import/raw/master/colabimport.py'
filename = colabimporturl.split("/")[-1].split("?")[0]
if os.path.isfile(filename):
    os.remove(filename)
wget.download(colabimporturl)

import colabimport
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/reusable_annotator.ipynb?raw=true')
colabimport.get_notebook('https://github.com/ratmcu/wiki_ner/blob/master/info_box.ipynb?raw=true')
# import io, os, sys, types
from reusable_annotator import PageContents
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
class InfoCard():
    '''accepts a beautiful soup html table from a wikipedia page and scrapes the table to a dictionary'''
    def __init__(self, page_content):
        self.info_table = {}
        if not page_content.table:
            raise Exception('page content is incomplete')
        for table_entry in page_content.table_entry_list:
            try:
                left_col = table_entry.find('th', attrs={'scope': 'row'})
                right_col = left_col.next_sibling
                self.info_table[self._row_filter(left_col.text)] = self._get_text_parts(self._get_kids(right_col))
            except:
                pass
        self.info_table_unfiltered = self.info_table
        self.info_table = self._filter_info_scrapes(self.info_table)
        print('info card is scraped successfully')
        logging.debug(self.info_table)
    
    def _row_filter(self, text):
        return re.sub('\xa0', ' ', text)
        
        
    def _get_kids(self, html_mother):
        kid_list = []
        try:
            kids = html_mother.children
            for kid in kids:
                kid_list.append(self._get_kids(kid))
        except:
            return html_mother
        return kid_list 

    def _get_text_parts(self, text_lists):
        text_parts = []
        if type(text_lists) == list:
            for element in text_lists:
                text_parts.extend(self._get_text_parts(element))
        else:
            text_parts.append(text_lists)
        return(text_parts)

    def _filter_info_scrapes(self, scape_dict):
        mask_dict = {}
        for key, val in scape_dict.items():
            mask_dict[key] = []
            for i, element in enumerate(scape_dict[key]):
                name = ''
                lst = [' '+nm[0] for nm in [part.split('(') for part in element.split(' ')] if nm[0]!='']
                element = ''.join(lst).strip()
                fnd = [c in element for c in u'[]\n\xa0']
                if not True in fnd:
                    mask_dict[key].append(element)  
        return mask_dict    

In [91]:
#experiment_code
# pg = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('http://en.wikipedia.org/wiki/Gajendra_Singh_Rajukhedi')

info_card = InfoCard(pg)
print(info_card.info_table_unfiltered)
print(info_card.info_table)


info card is scraped successfully
{'Vice President': ['Mike Pence'], 'Preceded by': ['Barack Obama'], 'Born': ['Donald John Trump', ' (', '1946-06-14', ') ', 'June 14, 1946', ' (age\xa073)', 'Queens', ', ', 'New York City'], 'Political party': ['Republican', ' (1987–1999, 2009–2011, 2012–present)'], 'Other politicalaffiliations': ['\n', 'Democratic', ' (until 1987, 2001–2009)', '\n', 'Reform', ' (1999–2001)', '\n', 'Independent', ' (2011–2012)', '\n'], 'Spouse(s)': ['\n', 'Ivana Zelníčková', '(', 'm.', '\xa0', '1977', '; ', 'div.', '\xa0', '1992', ')', '\n', 'Marla Maples', '(', 'm.', '\xa0', '1993', '; ', 'div.', '\xa0', '1999', ')', '\n', 'Melania Knauss', ' (', 'm.', '\xa0', '2005', ')', '\n'], 'Children': ['\n', 'Donald Jr.', '\n', 'Ivanka', '\n', 'Eric', '\n', 'Tiffany', '\n', 'Barron', '\n'], 'Parents': ['Fred Trump', 'Mary Anne MacLeod'], 'Relatives': ['Trump family'], 'Residence': ['\n', 'White House', ' (official)', '\n', 'Trump Tower', ' (personal)', '\n', 'Full list', '\n'],

In [0]:
# can we use snorkle to get this entity scraping done??? 
# if so it'd be better to have the lable functions in a ready way to generally working on all the pages!!

class PrivateEntities():
    '''given a dictionary of a side bar it is processed for private entities of interest'''
    def __init__(self, info_card):
        self.info_dict = info_card.info_table
        # print(info_card.info_table)
        self._get_entity_dict()
        self._extract_entities()
        
    def _extract_entities(self):
        # TREAT everyone the same? or call a seperate function per each entity?
        # leave the not found entities as empty lists so we can use the html scraper to fill them (usually the name of the person)
        # lets treat everything generally, ultimate filtering happens at the comparison stage(if theres one)
        for entity_key in self.entity_dict.keys():
            self._pick_entity(self.entity_dict[entity_key], entity_key)
            logging.debug(self.entity_dict[entity_key])
    
    def _pick_entity(self, entity_list, entity_key):
        nlp = spacy.load("en_core_web_sm")
#         print(entity_list)
#         Find named entities, phrases and concepts
        entity_list.append([])
#         print(entity_list[0][0])
        
        if entity_key == 'BIRTH_PLACE':
            mask_str = ''
            for scrape in self.info_dict[entity_list[0][0]]:
                doc = nlp(str(scrape))
                if doc.ents and doc.ents[0].label_ == 'DATE':
                    mask_str = mask_str + 'd'
                elif str(scrape) == '' or str(scrape) == ',':
                    mask_str = mask_str + 's'
                elif self._is_name(str(scrape)):
                    mask_str = mask_str + 't'
                else:
                    mask_str = mask_str + 'u'
#             print(mask_str)
            indices = self._get_target_indices([r'[s][t]',r'[d][t]'], mask_str)  
            for index in indices:
                entity_list[1].append(self.info_dict[entity_list[0][0]][index])
        else:    
            for scrape in self.info_dict[entity_list[0][0]]:
                if entity_list[0][1] == 'PERSON': #spacy skips certain parts of a name, so we use this trick instead
                    doc = nlp('was named '+str(scrape))
                    logging.debug(scrape)
                else:
                    doc = nlp(str(scrape))
                if doc.ents and doc.ents[0].label_ == entity_list[0][1]:   
                    entity_list[1].append(doc.ents[0].text)
                    logging.debug('appended entity: ', doc.ents[0].text, doc.ents[0].label_, len(doc.ents), doc.ents)
                elif not self._entity_noise(entity_key, str(scrape)):
                    entity_list[1].append(str(scrape))
                else:
                    pass
                
    def _get_target_indices(self, pattern_list, mask_str):
        '''TODO: if the target is inside the pattern'''
        indices = []
        for pattern in pattern_list:
            m_iter = re.finditer(pattern, mask_str)
            if m_iter:
                for m in m_iter: 
                    indices.append(m.start()+1)
        return indices        
        
    def _entity_noise(self, entity_key, text):
        try:
            return getattr(self, '_filter_{0}'.format(entity_key))(text)
        except:
            logging.error('filter function for {0} not found'.format(entity_key))
            return True
    
    def _get_entity_dict(self):
        self.entity_dict = {}
        if 'Born' in self.info_dict.keys():
            self.entity_dict['NAME'] = [['Born', 'PERSON']]
            self.entity_dict['BIRTH_DATE'] = [['Born', 'DATE']]
            self.entity_dict['BIRTH_PLACE'] = [['Born', 'GPE']]
        if 'Children' in self.info_dict.keys():
            self.entity_dict['CHILDREN'] = [['Children', 'PERSON']]
        if 'Spouse(s)' in self.info_dict.keys():
            self.entity_dict['SPOUSES'] = [['Spouse(s)', 'PERSON']]
        if 'Parents' in self.info_dict.keys():
            self.entity_dict['PARENTS'] = [['Parents', 'PERSON']]
        if 'Education' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] = [['Education', 'ORG']]
        if 'Alma mater' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] = [['Alma mater', 'ORG']]
    
    def _filter_NAME(self, text):
        '''cannot simply filter noise due to many possible candidates'''
        return True
    def _filter_BIRTH_DATE(self, text):
        return True
    def _filter_BIRTH_PLACE(self, text):
        return True
    def _filter_CHILDREN(self, text):
#         return False # let's assume we don't find any noisy text under info boxes children till we do an analysis
        return not self._is_name(text)
    def _filter_SPOUSES(self, text):
        return not self._is_name(text)
    def _filter_PARENTS(self, text):
        return not self._is_name(text)
    def _filter_EDUCATION(self, text):
        return True
    def _is_name(self, text):
        p = regex.compile(r"\p{Lu}") # To support (currently) 1702 uppercase letters
#         p = regex.compile(r"[[:upper:]]") # To support (currently) 1822 uppercase letters
        if p.match(text):
            return True
        else:
            return False
        


In [93]:
# pg = PageContents('https://en.wikipedia.org/wiki/James_R._Alexander')
# pg = PageContents('https://en.wikipedia.org/wiki/Brad_Pitt')
pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('https://en.wikipedia.org/wiki/Mohamed_Abdel-El')
info_card = InfoCard(pg)
entities = PrivateEntities(info_card).entity_dict
print(info_card.info_table['Born'], entities)

pg = PageContents('https://en.wikipedia.org/wiki/Brad_Pitt')
info_card = InfoCard(pg)
entities = PrivateEntities(info_card).entity_dict
print(info_card.info_table['Born'], entities)

pg = PageContents('http://en.wikipedia.org/wiki/Gajendra_Singh_Rajukhedi')
info_card = InfoCard(pg)
entities = PrivateEntities(info_card).entity_dict
print(info_card.info_table['Born'], entities)

info card is scraped successfully
['Donald John Trump', '', '1946-06-14', ')', 'June 14, 1946', '', 'Queens', ',', 'New York City'] {'NAME': [['Born', 'PERSON'], ['Donald John Trump', 'Queens']], 'BIRTH_DATE': [['Born', 'DATE'], ['1946-06-14', 'June 14, 1946']], 'BIRTH_PLACE': [['Born', 'GPE'], ['Queens', 'New York City']], 'CHILDREN': [['Children', 'PERSON'], ['Donald Jr.', 'Ivanka', 'Eric', 'Tiffany', 'Barron']], 'SPOUSES': [['Spouse(s)', 'PERSON'], ['Ivana Zelníčková', 'Marla Maples', 'Melania Knauss']], 'PARENTS': [['Parents', 'PERSON'], ['Fred Trump', 'Mary Anne MacLeod']], 'EDUCATION': [['Education', 'ORG'], ['The Wharton School']]}
info card is scraped successfully
['William Bradley Pitt', '', '1963-12-18', ')', 'December 18, 1963', '', 'Shawnee, Oklahoma', ', U.S.'] {'NAME': [['Born', 'PERSON'], ['William Bradley Pitt']], 'BIRTH_DATE': [['Born', 'DATE'], ['1963-12-18', 'December 18, 1963']], 'BIRTH_PLACE': [['Born', 'GPE'], ['Shawnee, Oklahoma']], 'CHILDREN': [['Children', 'PER

In [94]:
#experiment_code
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
# pg = PageContents('https://en.wikipedia.org/wiki/Bella_Hadid')
# pg = PageContents('https://en.wikipedia.org/wiki/Mahinda_Rajapaksa')
# pg = PageContents('https://en.wikipedia.org/wiki/Joachim_Gauck')
# pg = PageContents('https://en.wikipedia.org/wiki/Mike_Tyson')
pg = PageContents('https://en.wikipedia.org/wiki/Brad_Pitt')


info_card = InfoCard(pg)
# print(type(str(info_card.info_table['Born'][0])))
pe = PrivateEntities(info_card)
# pe.entity_dict

info card is scraped successfully


In [95]:
#experiment_code
info_card.info_table

{'Alma mater': ['University of Missouri'],
 'Born': ['William Bradley Pitt',
  '',
  '1963-12-18',
  ')',
  'December 18, 1963',
  '',
  'Shawnee, Oklahoma',
  ', U.S.'],
 'Children': ['6'],
 'Education': ['Kickapoo High School'],
 'Home town': ['Springfield, Missouri', ', U.S.'],
 'Net worth': ['$240 million'],
 'Occupation': ['Actor', 'film producer'],
 'Relatives': ['Douglas Pitt', ''],
 'Spouse(s)': ['Jennifer Aniston',
  '',
  'm.',
  '2000;',
  'div.',
  '2005)',
  'Angelina Jolie',
  '',
  'm.',
  '2014;',
  'div.',
  '2019)'],
 'Works': ['Filmography'],
 'Years active': ['1987–present']}

In [96]:
fz = fuzzyset.FuzzySet(use_levenshtein=False)
fz.add('Alma\xa0mater')        
fz.add('Alma ma')        
result = fz.get('Alma mater')
result
# [(0.7378647873726218, 'Almamater')]
re.search('\xa0', 'Alma\xa0mater').span(0)

(4, 5)

In [97]:
import re
ss = 'Gayatri Singh ('
name = ''
lst = [' '+nm[0] for nm in [part.split('(') for part in ss.split(' ')] if nm[0]!='']
# lst = [' '+nm[0] for nm in [part in part for re.split(' ()',ss)] if nm[0]!='']
# [part in part for re.split(' ()',ss)]
name.join(lst).strip()

'Gayatri Singh'